# Datasets

A [Dataset](https://nexus-forge.readthedocs.io/en/latest/interaction.html#dataset) is a specialization of a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that provides users with operations to handle files, record their provenance and describe them with metadata.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

Note: DemoStore doesn't implement file operations yet. Use the BluBrainNexus store instead when creating a config file.

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

In [4]:
from kgforge.specializations.resources import Dataset

In [5]:
import pandas as pd

## Creation with files

In [6]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [7]:
persons = Dataset(forge, name="Interesting Persons")

In [8]:
persons.add_files("../../data/persons.csv")

In [9]:
forge.register(persons)

<action> _register_one
<succeeded> True


In [10]:
forge.as_json(persons)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/8ceeffe4-02b2-4b58-b9dc-a131ec24c127',
 'type': 'Dataset',
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault'}},
   'contentSize': {'unitCode': 'bytes', 'value': 52},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/e44b3aa4-2445-4531-bba3-277983417495',
   'digest': {'algorithm': 'SHA-256',
    'value': '1dacd765946963fda4949753659089c5f532714b418d30788bedddfec47a389f'},
   'encodingFormat': 'text/csv',
   'name': 'persons.csv'}},
 'name': 'Interesting Persons'}

In [15]:
associations = Dataset(forge, name="Associations data")

In [16]:
associations.add_files("../../data/associations.tsv")

In [17]:
associations.add_derivation(persons)

In [18]:
forge.register(associations)

<action> _register_one
<succeeded> True


In [19]:
forge.as_json(associations)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/0c83bec0-a0e9-42d9-b489-6ed350536a93',
 'type': 'Dataset',
 'derivation': {'type': 'Derivation',
  'entity': {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/8ceeffe4-02b2-4b58-b9dc-a131ec24c127?rev=1',
   'type': 'Dataset',
   'name': 'Interesting Persons'}},
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault'}},
   'contentSize': {'unitCode': 'bytes', 'value': 477},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/ef481bf4-6378-46be-a352-3abd21f6bb56',
   'digest': {'algorithm': 'SHA-256',
    'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
   'encodingFormat': 'text/tab-separated-values',
   'name': 'associations.tsv'}},
 'name': 'Associations data'}

In [23]:
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.

# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
associations.download(path="./downloaded/", source="parts")

In [24]:
! ls -l ./downloaded

total 8
-rw-r--r--  1 mfsy  staff  477 Feb  9 12:32 associations.tsv


In [25]:
# ! rm -R ./downloaded/

## Creation with resources as parts

In [26]:
distribution_1 = forge.attach("../../data/associations.tsv")

In [27]:
distribution_2 = forge.attach("../../data/persons.csv")

In [28]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution_1)

In [29]:
john = Resource(type="Person", name="John Smith", distribution=distribution_2)

In [30]:
persons = [jane, john]

In [31]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [32]:
dataset = Dataset(forge, name="Interesting people")

In [33]:
dataset.add_parts(persons)

In [34]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [36]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/b1ec4926-e4e7-46ca-91bf-5cfc5e56d6ee',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/d4241163-5b8e-4806-8b75-c4fd3701a94d?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/93fce163-1103-4b76-a474-6de37a2592bf'},
   'name': 'Jane Doe'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/dabe7414-eb38-45a5-a3da-59dc500ae97a?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/01457a40-8551-43cd-be44-fb86733c8073'},
   'name': 'John Smith'}],
 'name': 'Interesting people'}

In [37]:
dataset.download(path="./downloaded/", source="parts")

In [38]:
! ls -l ./downloaded

total 24
-rw-r--r--  1 mfsy  staff  477 Feb  9 12:32 associations.tsv
-rw-r--r--  1 mfsy  staff  477 Feb  9 12:34 associations.tsv.20220209123401
-rw-r--r--  1 mfsy  staff   52 Feb  9 12:34 persons.csv


In [36]:
# ! rm -R ./downloaded/

### Creation from resources

In [39]:
dataset = Dataset.from_resource(forge, [jane, john], store_metadata=True)
print(*dataset, sep="\n")

{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/d4241163-5b8e-4806-8b75-c4fd3701a94d
    type: Person
    distribution:
    {
        type: DataDownload
        atLocation:
        {
            type: Location
            store:
            {
                id: https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault
            }
        }
        contentSize:
        {
            unitCode: bytes
            value: 477
        }
        contentUrl: https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/93fce163-1103-4b76-a474-6de37a2592bf
        digest:
        {
            algorithm: SHA-256
            value: 789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859
        }
        encodingFormat: text/tab-separated-values
        name: associations.tsv
    }
    name: Jane Doe
}
{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/dabe7414-eb38-45a5-a3da-59dc500ae97a
    type: Person
    distribution:
  

## Creation from a dataframe

See notebook `07 DataFrame IO.ipynb` for details on conversions of instances of Resource from a Pandas DataFrame.

### basics

In [40]:
dataframe = pd.read_csv("../../data/persons.csv")

In [41]:
dataframe

,type,name
0,Person,Marie Curie
1,Person,Albert Einstein


In [42]:
persons = forge.from_dataframe(dataframe)

In [43]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [44]:
dataset = Dataset(forge, name="Interesting people")

In [45]:
dataset.add_parts(persons)

In [46]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [47]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/d14fa3c8-684f-4ff2-abf6-1bafe7aed9c0',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/dfd3c1a2-b9e1-47f1-ad47-854a097c02e4?rev=1',
   'type': 'Person',
   'name': 'Marie Curie'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/52ac438a-8b55-4b22-af7b-5dce8a3e0c6d?rev=1',
   'type': 'Person',
   'name': 'Albert Einstein'}],
 'name': 'Interesting people'}

### advanced

In [48]:
dataframe = pd.read_csv("../../data/associations.tsv", sep="\t")

In [49]:
dataframe

,id,name,type,agent__type,agent__name,agent__gender__id,agent__gender__type,agent__gender__label,distribution
0,(missing),Curie Association,Association,Person,Marie Curie,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,../../data/scientists-database/marie_curie.txt
1,(missing),Einstein Association,Association,Person,Albert Einstein,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,../../data/scientists-database/albert_einstein...


In [50]:
dataframe["distribution"] = dataframe["distribution"].map(lambda x: forge.attach(x))

In [51]:
associations = forge.from_dataframe(dataframe, na="(missing)", nesting="__")

In [52]:
print(*associations, sep="\n")

{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000383
            type: LabeledOntologyEntity
            label: female
        }
        name: Marie Curie
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/marie_curie.txt', None])
    name: Curie Association
}
{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000384
            type: LabeledOntologyEntity
            label: male
        }
        name: Albert Einstein
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/albert_einstein.txt', None])
    name: Einstein Association
}


In [53]:
forge.register(associations)

<count> 2
<action> _register_many
<succeeded> True


In [54]:
dataset = Dataset(forge, name="Interesting associations")

In [55]:
dataset.add_parts(associations)

In [56]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [57]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/b5897b4b-c2d5-48d8-a4bb-fcbfcb9dab25',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/2d6cdb1a-eb10-4d7f-87ad-6afc6b3ad6fb?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/b46b854f-ef72-4a5c-a6b0-cccf8748d35b'},
   'name': 'Curie Association'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/87acccee-75eb-493d-afa6-bf3a8ec3f5a8?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/26b57bff-9828-4696-90a5-e5229597c70a'},
   'name': 'Einstein Association'}],
 'name': 'Interesting associations'}